In [ ]:
import blocksci

import sys, os, os.path, socket
import numpy as np
import zarr
import time
import pandas as pd
from collections import defaultdict
from decimal import Decimal
import datetime

def format_e(n):
    a = '%E' % n
    return a.split('E')[0].rstrip('0').rstrip('.') + 'E' + a.split('E')[1]

# Counting number of True and False values in cluster_is_black_ground_truth 

In [ ]:
array_zarr = zarr.load('uniform_black/heur_1_data/cluster_is_black_ground_truth.zarr') #load zarr file into array
#df = pd.DataFrame (array_zarr) ## convert your array into a dataframe
#filepath = 'zarr_to_excel.csv'
#df.to_csv(filepath, index=False)
countTrue = np.where(array_zarr==True)
countFalse = np.where(array_zarr==False)
element_count_False = countFalse[0].size
element_count_True = countTrue[0].size
print(len(array_zarr))
print(element_count_False)
print(element_count_True)
print(array_zarr)
for i in countTrue:
    print(i)

# Importing saved results NPY vs ZARR:

In [ ]:
for i in range(49990,49996):
    # import npy and place in dict
    start_time = datetime.datetime.now()
    P = np.load(f'/local/scratch/exported/blockchain_parsed/bitcoin_darknet/gs_group/grayscale_op_ali/heur_1_data/current_assets/current_assets_block_{str(i).zfill(6)}.npy', allow_pickle=True) #load gives an object array containing this dictionary:
    current_assets =defaultdict(lambda: np.zeros(2))
    current_assets.update(P.item())
    end_time = datetime.datetime.now()
    print(end_time - start_time)

    print(f'current_assets length')
    print(len(current_assets))

    start_time = datetime.datetime.now()
    # import zarr and place in dict
    current_assets_zarr = zarr.load(f'/local/scratch/exported/blockchain_parsed/bitcoin_darknet/gs_group/grayscale_op_ali/heur_1_data_zarr/current_assets/current_assets_values_block_{str(i).zfill(6)}.zarr') #load gives an object array containing this dictionary:
    current_assets_index_zarr = zarr.load(f'/local/scratch/exported/blockchain_parsed/bitcoin_darknet/gs_group/grayscale_op_ali/heur_1_data_zarr/current_assets_index/current_assets_index_block_{str(i).zfill(6)}.zarr')
    current_assets_zarr = dict(zip(current_assets_index_zarr, current_assets_zarr))
    end_time = datetime.datetime.now()
    print(end_time - start_time)
    print(f'current_assets_zarr length')
    print(len(current_assets_zarr))

    for key in current_assets.keys() & current_assets_zarr.keys():
        value1 = current_assets[key]
        value2 = current_assets_zarr[key]
        if value1 != value2:
            print("error")
    
    # i = 0
    # for k, v in current_assets_zarr.items():
    #     print(f'{k}:{format_e(Decimal(v))}, ', end='')


    # i = 0
    # for k, v in current_assets.items():
    #     if v != 0.0 :
    #         print(f'{k}:{format_e(Decimal(v))}, ', end='')
        # if i == 9:
        #     print('\n')
        #     i = 0
        # i+=1

# Importing saved results daily :

In [ ]:

current_assets_zarr = zarr.load(f'/local/scratch/exported/blockchain_parsed/bitcoin_darknet/gs_group/grayscale_op_ali/heur_1_data_zarr_day/current_assets/current_assets_values_2010-06-12.zarr') #load gives an object array containing this dictionary:
current_assets_index_zarr = zarr.load(f'/local/scratch/exported/blockchain_parsed/bitcoin_darknet/gs_group/grayscale_op_ali/heur_1_data_zarr_day/current_assets_index/current_assets_index_2010-06-12.zarr')
current_assets_zarr = dict(zip(current_assets_index_zarr, current_assets_zarr))
print(len(current_assets_zarr))
i = 0
for k, v in current_assets_zarr.items():
    print(f'{k}:{format_e(v)}, ', end='')
    # if i == 9:
    #     print('\n')
    #     i = 0
    # i+=1

# Importing saved results per block:


In [ ]:
for i in range(49995,49996):
    current_assets_zarr = zarr.load(f'/local/scratch/exported/blockchain_parsed/bitcoin_darknet/gs_group/grayscale_op_ali/heur_1_data_zarr/current_assets/current_assets_values_block_{str(i).zfill(6)}.zarr') #load gives an object array containing this dictionary:
    current_assets_index_zarr = zarr.load(f'/local/scratch/exported/blockchain_parsed/bitcoin_darknet/gs_group/grayscale_op_ali/heur_1_data_zarr/current_assets_index/current_assets_index_block_{str(i).zfill(6)}.zarr')
    print(f'current_assets before')
    print(len(current_assets_zarr))
    i = 0
    for v, k in zip(current_assets_zarr, current_assets_index_zarr):
        print(f'{k}:{format_e(Decimal(v))}, ', end='')
        # if i == 9:
        #     print('\n')
        #     i = 0
        # i+=1

# Gray Scale Model

### Important Variables:

In this task we will be implementing the SIR model with the following system parameters:

*   $a_{i}(b)$: total amount of assets hold by user i by the end of block b.
*   $d_{i}(b)$: total amount of black assets hold by node i by the end of block b.
*   $x_{i}(b)$: = $\frac{d_{i}(b)}{a_{i}(b)} $ black assets fraction by the end of block b.
*   $c_{i}(b)$: amount of clean asset owned by node i by the end of block b
*   $m_{i}(b)$: the amount mined by user i by the end of block b
*   $W(b)$: = {$w_{ij}$}$_{i,j=1,...,N}$ is the adjacency matrix of the transaction network at block b. wij is equal to the amount of bitcoin moved from user i to user j at block b.
*   $N = Nusers = |Nusers|$ total number of users in the network.

Below is a summary of the dynamics of the SIR model: